In [2]:
!pip install --disable-pip-version-check -q awswrangler==2.14.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-expr 1.0.11 requires pandas>=2, but you have pandas 1.3.5 which is incompatible.
dask-expr 1.0.11 requires pyarrow>=7.0.0, but you have pyarrow 6.0.1 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [3]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

In [4]:
import boto3
import sagemaker
import pandas as pd
from pyathena import connect

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
s3 = boto3.resource('s3')

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
s3_private_path_csv = "s3://{}/dataset/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-652903355321/dataset/csv


In [6]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
print(s3_staging_dir)

s3://sagemaker-us-east-1-652903355321/athena/staging


In [7]:
import os
# Get the current working directory
current_directory = os.getcwd()
# Specify the relative path to the data file within the current directory
file_path = os.path.join(current_directory, 'dataset.csv')
print("Path of the data file:", file_path)

Path of the data file: /root/aai-540-labs/MLOpsFinalProject/dataset.csv


In [8]:
s3.Object(bucket, 'dataset/csv/dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/dataset.csv', 'rb'))

{'ResponseMetadata': {'RequestId': 'PPQ8VEWP8JNYNAA0',
  'HostId': 'Nirj6usAmiI/eCY66/Zhxi37WuNUBbr+D7ywKSBjfw+1ulZmFMNjCB5WqoC4myD3Ptp1AlEU4zg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Nirj6usAmiI/eCY66/Zhxi37WuNUBbr+D7ywKSBjfw+1ulZmFMNjCB5WqoC4myD3Ptp1AlEU4zg=',
   'x-amz-request-id': 'PPQ8VEWP8JNYNAA0',
   'date': 'Sun, 12 May 2024 23:36:35 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e50a70711e823b75c8ebe231468f5faf"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e50a70711e823b75c8ebe231468f5faf"',
 'ServerSideEncryption': 'AES256'}

In [9]:
!aws s3 ls $s3_private_path_csv/

2024-05-12 23:36:35   20118244 dataset.csv


In [10]:
!aws s3 ls s3://sagemaker-us-east-1-652903355321

                           PRE FinalProjectDataset/
                           PRE amazon-reviews-pds/
                           PRE athena/
                           PRE dataset/
2024-05-11 15:42:49   20118244 dataset.csv


In [11]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/amazon-reviews-pds/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

/tmp/ipykernel_18/4286372116.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [12]:
ingest_create_athena_db_passed = False

In [102]:
database_name = "Assignment2"
table_name = "MusicData"
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS Assignment2


In [103]:
import pandas as pd
pd.read_sql(statement, conn)

""


In [104]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,assgnaws
1,assignment2
2,default
3,dsoaws


In [105]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         srnbr int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit string,
         danceability string,
         energy string,
         key int,
         loudness string,
         mode int,
         speechiness string,
         acousticness string,
         instrumentalness string,
         liveness string,
         valence string,
         tempo string,
         time_signature int,
         track_genre string
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"'
) LOCATION '{}' 
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path_csv
)

print(statement)
pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS Assignment2.MusicData(
         srnbr int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit string,
         danceability string,
         energy string,
         key int,
         loudness string,
         mode int,
         speechiness string,
         acousticness string,
         instrumentalness string,
         liveness string,
         valence string,
         tempo string,
         time_signature int,
         track_genre string
) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES (
    'separatorChar' = ',',
    'quoteChar' = '"'
) LOCATION 's3://sagemaker-us-east-1-652903355321/dataset/csv' 
TBLPROPERTIES ('skip.header.line.count'='1')


""


In [108]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,homeworkdata_parquet
1,homeworkdataset
2,homeworkdataset_parquet
3,musicdata
4,musicdataset


In [109]:
statement = """SELECT DISTINCT track_genre FROM {}.{}
     LIMIT 100""".format(
    database_name, table_name
)

print(statement)

SELECT DISTINCT track_genre FROM Assignment2.MusicData
     LIMIT 100


In [110]:
df = pd.read_sql(statement, conn)
df.head(5)

,track_genre
0,acoustic
1,afrobeat
2,alt-rock
3,alternative
4,ambient


In [112]:
statement = """SELECT * FROM {}.{} LIMIT 100""".format(
    database_name, table_name
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM Assignment2.MusicData LIMIT 100


,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.42,0.166,...,-17.235,1,0.0763,0.924,5.56e-06,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.21,0.0,0.117,0.12,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic


In [113]:
track_genre = "acoustic"

statement = """SELECT * FROM {}.{}
    WHERE track_genre = '{}' LIMIT 100""".format(
    database_name, table_name, track_genre
)

print(statement)

SELECT * FROM Assignment2.MusicData
    WHERE track_genre = 'acoustic' LIMIT 100


In [114]:
df = pd.read_sql(statement, conn)
df.head(5)

,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.42,0.166,...,-17.235,1,0.0763,0.924,5.56e-06,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.21,0.0,0.117,0.12,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic


In [119]:
# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/dataset/parquet".format(bucket)

# Set Athena parameters
table_name_parquet = "MusicData1_parquet"

In [120]:
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['track_genre']) AS
SELECT srnbr,
        track_id,
        artists,
        album_name,
        track_name,
        popularity,
        duration_ms,
        explicit,
        danceability,
        energy,
        key,
        loudness,
        mode,
        speechiness,
        acousticness,
        instrumentalness,
        liveness,
        valence,
        tempo,
        time_signature,
        track_genre
FROM {}.{} WHERE srnbr <25000 """.format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name
)

print(statement)

CREATE TABLE IF NOT EXISTS Assignment2.MusicData1_parquet
WITH (format = 'PARQUET', external_location = 's3://sagemaker-us-east-1-652903355321/dataset/parquet', partitioned_by = ARRAY['track_genre']) AS
SELECT srnbr,
        track_id,
        artists,
        album_name,
        track_name,
        popularity,
        duration_ms,
        explicit,
        danceability,
        energy,
        key,
        loudness,
        mode,
        speechiness,
        acousticness,
        instrumentalness,
        liveness,
        valence,
        tempo,
        time_signature,
        track_genre
FROM Assignment2.MusicData WHERE srnbr <25000 


In [121]:
import pandas as pd

pd.read_sql(statement, conn)

,rows


In [122]:
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)
print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

MSCK REPAIR TABLE Assignment2.MusicData1_parquet


""


In [123]:
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_name_parquet)
print(statement)
df_partitions = pd.read_sql(statement, conn)
df_partitions.head(5)

SHOW PARTITIONS Assignment2.MusicData1_parquet


,partition
0,track_genre=deep-house
1,track_genre=cantopop
2,track_genre=breakbeat
3,track_genre=country
4,track_genre=death-metal


In [124]:
statement = "SHOW TABLES in {}".format(database_name)
df_tables = pd.read_sql(statement, conn)
df_tables.head(5)

,tab_name
0,homeworkdata_parquet
1,homeworkdataset
2,homeworkdataset_parquet
3,musicdata
4,musicdata1_parquet


In [125]:
track_genre = "acoustic"

statement = """SELECT * FROM {}.{}
    WHERE track_genre = '{}' LIMIT 100""".format(
    database_name, table_name_parquet, track_genre
)

print(statement)
df_tables = pd.read_sql(statement, conn)
df_tables.head(5)

SELECT * FROM Assignment2.MusicData1_parquet
    WHERE track_genre = 'acoustic' LIMIT 100


,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.42,0.166,...,-17.235,1,0.0763,0.924,5.56e-06,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.21,0.0,0.117,0.12,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic


In [126]:
import awswrangler as wr

In [132]:
# Query Data with AWS Data Wrangler

In [133]:
p_filter = lambda x: x["track_genre"] == "acoustic"

In [134]:
df_parquet_results = wr.s3.read_parquet(
    s3_path_parquet, columns=["track_name", "track_genre", "artists"], partition_filter=p_filter, dataset=True
)
df_parquet_results.shape

(1000, 3)

In [135]:
df_parquet_results.head(5)

,track_name,artists,track_genre
0,Comedy,Gen Hoshino,acoustic
1,Ghost - Acoustic,Ben Woodward,acoustic
2,To Begin Again,Ingrid Michaelson;ZAYN,acoustic
3,Can't Help Falling In Love,Kina Grannis,acoustic
4,Hold On,Chord Overstreet,acoustic


In [138]:
chunk_iter = wr.s3.read_parquet(
    s3_path_parquet,
    columns=["track_name", "track_genre", "artists"],  
    partition_filter=p_filter,
    dataset=True,
    chunked=True,
)

In [139]:
print(next(chunk_iter))

                             track_name                 artists track_genre
0                                Comedy             Gen Hoshino    acoustic
1                      Ghost - Acoustic            Ben Woodward    acoustic
2                        To Begin Again  Ingrid Michaelson;ZAYN    acoustic
3            Can't Help Falling In Love            Kina Grannis    acoustic
4                               Hold On        Chord Overstreet    acoustic
..                                  ...                     ...         ...
995                               Maria              Harley Poe    acoustic
996  Corinna - From "The Natch'l Blues"               Taj Mahal    acoustic
997                           Tall Cans              Pigeon Pit    acoustic
998               You Get What You Give             Chuck Ragan    acoustic
999                   Four Simple Words            Frank Turner    acoustic

[1000 rows x 3 columns]


In [140]:
%%time
df = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5000".format(table_name_parquet), database=database_name)

CPU times: user 676 ms, sys: 22.5 ms, total: 699 ms
Wall time: 3.67 s


In [141]:
df.head(5)

,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,20000,5IgjP7X4th6nMNDh4akUHb,Chris Brown,Indigo (Extended),Under The Influence,96,184613,True,0.733,0.69,...,-5.529,0,0.0427,0.0635,1.18e-06,0.105,0.31,116.992,4,dance
2,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
3,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic
4,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.481,...,-8.807,1,0.105,0.289,0.0,0.189,0.666,98.017,4,acoustic


In [142]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql="SELECT * FROM {} LIMIT 5000".format(table_name_parquet),
    database="{}".format(database_name),
    chunksize=64_000,  # 64 KB Chunks
)

CPU times: user 337 ms, sys: 9.37 ms, total: 346 ms
Wall time: 3.03 s


In [143]:
print(next(chunk_iter))

      srnbr                track_id                   artists  \
0      3000  5E30LdtzQTGqRvNd7l6kG5         The Neighbourhood   
1      3001  09gysnJpfQ3ublBmJDfcEC  Amaarae;Kali Uchis;Moliy   
2      3002  2K7xn816oNHJZ0aVqdQsha         The Neighbourhood   
3      3003  2QjOHCTQ1Jl3zawyYOpxh6         The Neighbourhood   
4      3004  4fouWK6XVHhzl78KzQ1UjL                     GAYLE   
...     ...                     ...                       ...   
4995  19927  6oDihOuvOytAzuQt3SMDHk                Koe Wetzel   
4996  19928  2PGKzBxAvi1KOn9pSpHZcx                  Upchurch   
4997  19929  78vTiLFfciZ5kiryzaWFbk             Vassendgutane   
4998  19930  2tEMqkUo0QCtGOyUATJRxZ                  Upchurch   
4999  19931  6h5hwJ9ZfFZ81JqBjhgLwe              Too Far Gone   

                                             album_name  \
0                                            Wiped Out!   
1     SAD GIRLZ LUV MONEY Remix (feat. Kali Uchis an...   
2       Hard To Imagine The Neighbourhood 

In [26]:
#path = "s3://{}/amazon-reviews-pds/parquet/".format(bucket)
df_csv_results = wr.s3.read_csv(s3_private_path_csv, dataset=True)
df_csv_results.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [144]:
%%time
df_wr_athena = wr.athena.read_sql_query(sql="SELECT * FROM {} LIMIT 5000".format(table_name), database=database_name)
df_wr_athena.head(5)

CPU times: user 364 ms, sys: 24.2 ms, total: 388 ms
Wall time: 2.89 s


,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.42,0.166,...,-17.235,1,0.0763,0.924,5.56e-06,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.21,0.0,0.117,0.12,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic


In [145]:
track_genre="children"
df_wr_athena = wr.athena.read_sql_query(sql="SELECT * FROM {} WHERE track_genre = '{}' LIMIT 100".format(table_name,track_genre ), database=database_name)
df_wr_athena.head(5)

,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,14000,5ygDXis42ncn6kYG14lEVG,Pinkfong,Pinkfong Animal Songs,Baby Shark,77,80927,False,0.829,0.886,...,-1.746,1,0.112,0.259,0.0,0.0559,0.777,115.056,4,children
1,14001,6YQ7aPJhk0MGpwoKfFAEbS,CoComelon,"CoComelon Kids Hits, Vol. 1",Wheels on the Bus,72,207340,False,0.941,0.387,...,-11.92,1,0.0427,0.184,2.88e-05,0.157,0.965,125.021,4,children
2,14002,4UPrf1sQYOG2TIBScms919,LooLoo Kids,"The Best Songs for Kids, Vol. 1",Johny Johny Yes Papa,46,94451,False,0.597,0.764,...,-6.034,1,0.0318,0.0551,0.000764,0.249,0.786,154.98,4,children
3,14003,7zD97mcPo9fng0aJqUhFvd,CoComelon,Nursery Rhymes by CoComelon,Baby Shark,64,127203,False,0.779,0.587,...,-8.131,1,0.0721,0.33,3.85e-06,0.0792,0.908,199.956,4,children
4,14004,3N6kzbnfpTPB5J9NAGc1rU,Super Simple Songs,Twinkle Twinkle Little Star & More Kids Songs,Twinkle Twinkle Little Star,69,152691,False,0.696,0.227,...,-11.763,1,0.042,0.866,1.25e-05,0.0794,0.284,87.88,4,children


In [146]:
track_genre="children"
popularity = 60
df_wr_athena = wr.athena.read_sql_query(sql="SELECT * FROM {} WHERE track_genre = '{}' AND popularity >={} ".format(table_name,track_genre ,popularity), database=database_name)
df_wr_athena

,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,14000,5ygDXis42ncn6kYG14lEVG,Pinkfong,Pinkfong Animal Songs,Baby Shark,77,80927,False,0.829,0.886,...,-1.746,1,0.112,0.259,0.0,0.0559,0.777,115.056,4,children
1,14001,6YQ7aPJhk0MGpwoKfFAEbS,CoComelon,"CoComelon Kids Hits, Vol. 1",Wheels on the Bus,72,207340,False,0.941,0.387,...,-11.92,1,0.0427,0.184,2.88e-05,0.157,0.965,125.021,4,children
2,14003,7zD97mcPo9fng0aJqUhFvd,CoComelon,Nursery Rhymes by CoComelon,Baby Shark,64,127203,False,0.779,0.587,...,-8.131,1,0.0721,0.33,3.85e-06,0.0792,0.908,199.956,4,children
3,14004,3N6kzbnfpTPB5J9NAGc1rU,Super Simple Songs,Twinkle Twinkle Little Star & More Kids Songs,Twinkle Twinkle Little Star,69,152691,False,0.696,0.227,...,-11.763,1,0.042,0.866,1.25e-05,0.0794,0.284,87.88,4,children
4,14005,0dF7HP4wDwxCtID3mXpdYX,CoComelon,"CoComelon Kids Hits, Vol. 1",Baa Baa Black Sheep,68,147780,False,0.711,0.396,...,-11.266,1,0.0486,0.829,3e-05,0.119,0.969,199.955,4,children
5,14006,5txSLkf7BeYIgTtUnVDDG8,CoComelon,"CoComelon Kids Hits, Vol. 1",Old MacDonald,64,147147,False,0.704,0.479,...,-12.362,1,0.0348,0.292,1.14e-06,0.0419,0.936,199.841,4,children
6,14007,1nTBgueuGtO762MadGoWvZ,CoComelon,"CoComelon Kids Hits, Vol. 2",Five Little Monkeys Jumping on the Bed,64,169302,False,0.959,0.2,...,-12.65,1,0.155,0.8,0.00398,0.0842,0.717,119.925,3,children
7,14008,2R5iTxkCln6NLTw6l0kjWw,CoComelon,"CoComelon Kids Hits, Vol. 3",Happy Birthday Song,62,62228,False,0.866,0.387,...,-11.011,1,0.0345,0.403,0.0,0.145,0.79,94.976,3,children
8,14010,6xfOJl9l0Gc5WxjmBAvQsA,CoComelon,CoComelon Essentials,Head Shoulder Knees and Toes,64,127026,False,0.516,0.351,...,-11.012,1,0.044,0.709,8.61e-05,0.116,0.796,159.757,4,children
9,14012,097ftnCPnGHhn93lS0HHlO,CoComelon,"CoComelon Kids Hits, Vol. 3",Twinkle Twinkle Little Star,67,177477,False,0.459,0.206,...,-9.694,1,0.0282,0.673,0.0,0.294,0.354,160.072,4,children


In [147]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>